In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Dataset

In [2]:
class EmotionSequenceDataset(Dataset):
    def __init__(self, df):
        self.df = df.copy()
        self.df['deceptive'] = self.df['deceptive'].astype(int)
        self.features_cols = df.columns.difference(['id', 'frame', 'deceptive'])
        self.sample_ids = self.df['id'].unique().tolist()

    def __len__(self):
        return len(self.sample_ids)

    def __getitem__(self, idx):
        sample_id = self.sample_ids[idx]
        subset = self.df[self.df['id'] == sample_id]
        X = subset[self.features_cols].values.astype(np.float32)
        y = subset['deceptive'].iloc[0]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
    
def collate_fn(batch):
    sequences, labels = zip(*batch)
    lengths = [len(seq) for seq in sequences]
    padded_sequences = nn.utils.rnn.pad_sequence(sequences, batch_first=True)
    return padded_sequences, torch.tensor(lengths), torch.tensor(labels, dtype=torch.float32)

# BiLSTM with attention

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim * 2, 1)

    def forward(self, lstm_out, mask):
        scores = self.attn(lstm_out).squeeze(-1)
        scores = scores.masked_fill(~mask, -1e4)
        weights = torch.softmax(scores, dim=1).unsqueeze(-1)
        weights = torch.softmax(scores, dim=1).unsqueeze(-1)
        weights = torch.nan_to_num(weights, nan=0.0)
        context = torch.sum(weights * lstm_out, dim=1)
        return context, weights

class BiLSTMAttention(nn.Module):
    def __init__(self, input_dim=967, hidden_dim=64, num_layers=2, proj_dim=128, dropout=0.3):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, proj_dim)
        nn.init.xavier_uniform_(self.input_proj.weight)
        nn.init.zeros_(self.input_proj.bias)
        self.layer_norm = nn.LayerNorm(proj_dim)
        self.lstm = nn.LSTM(proj_dim, hidden_dim, num_layers=num_layers,
                            bidirectional=True, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.attn = Attention(hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, x, lengths):
        x = self.input_proj(x)
        x = self.layer_norm(x)
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        out, _ = self.lstm(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)

        mask = torch.arange(out.size(1), device=out.device)[None, :] < lengths[:, None].to(out.device)
        context, weights = self.attn(out, mask)
        context = self.dropout(context)
        logits = self.fc(context).squeeze(1)

        return logits, weights

In [ ]:
df = pd.read_csv("processed_data/silesian_deception_dataset/emotions_landmarks_flow.csv")
bad_rows = df[df.isna().any(axis=1)]
bad_ids = bad_rows['id'].unique().tolist()
df = df[~df['id'].isin(bad_ids)].reset_index(drop=True)
print(max([len(df[df['id']==sid]) for sid in df['id'].unique()]))
print(min([len(df[df['id']==sid]) for sid in df['id'].unique()]))

samples = df['id'].unique().tolist()
train_samples, test_samples = train_test_split(samples, test_size=0.2, random_state=42)
train_df = df[df['id'].isin(train_samples)]
test_df = df[df['id'].isin(test_samples)]

landmarks_cols = [f'lm_{i}' for i in range(468*2)]
flow_cols = ['flow_mean_x', 'flow_mean_y', 'flow_std_x', 'flow_std_y']
emotion_cols = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
scalers = {}
for cols in [landmarks_cols, flow_cols, emotion_cols]:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    train_df[cols] = scaler.fit_transform(train_df[cols])
    test_df[cols] = scaler.transform(test_df[cols])
    scalers.update({col: scaler for col in cols})

train_dataset = EmotionSequenceDataset(train_df)
test_dataset = EmotionSequenceDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn, shuffle=False, drop_last=True)

print(f"Global Class Distribution: {df['deceptive'].value_counts().to_dict()}")
print(f"Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")
# df.head()

670
122


/tmp/ipykernel_4438/83718974.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[features_cols] = scaler.fit_transform(train_df[features_cols])
/tmp/ipykernel_4438/83718974.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[features_cols] = scaler.transform(test_df[features_cols])


Global Class Distribution: {True: 128876, False: 61071}
Train samples: 733, Test samples: 184


In [5]:
num_pos = (train_df['deceptive'] == 1).sum()
num_neg = (train_df['deceptive'] == 0).sum()
print(f"Number of positive samples: {num_pos}, Number of negative samples: {num_neg}")
pos_weight = torch.tensor(num_neg / num_pos, dtype=torch.float32).to(device)

Number of positive samples: 102194, Number of negative samples: 49883


# Training

In [8]:
model = BiLSTMAttention().to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

for epoch in range(10):
    model.train()
    total_loss = 0
    for X, lengths, y in train_loader:
        X = X.to(device)
        y = y.to(device)
        opt.zero_grad()
        logits, _ = model(X, lengths)
        loss = criterion(logits, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, loss={total_loss/len(train_loader):.4f}")

Epoch 1, loss=0.4586
Epoch 2, loss=0.4551
Epoch 3, loss=0.4519
Epoch 4, loss=0.4535
Epoch 5, loss=0.4516
Epoch 6, loss=0.4528
Epoch 7, loss=0.4533
Epoch 8, loss=0.4526
Epoch 9, loss=0.4502
Epoch 10, loss=0.4506


In [9]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X, lengths, y in test_loader:
        X = X.to(device)
        y = y.to(device)

        logits, _ = model(X, lengths)
        preds = torch.sigmoid(logits)
        preds = (preds > 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
cm = confusion_matrix(all_labels, all_preds)

print(f"✅ Test accuracy: {acc:.4f}")
print(f"✅ F1 score: {f1:.4f}")
print("✅ Confusion matrix:")
print(cm)

✅ Test accuracy: 0.7065
✅ F1 score: 0.8258
✅ Confusion matrix:
[[  2  52]
 [  2 128]]


# Purely emotion-based dataset analysis

In [ ]:
df = pd.read_csv("processed_data/silesian_deception_dataset/emotions.csv")
df.groupby("deceptive").mean()

,id,frame,Angry,Disgust,Fear,Happy,Sad,Surprise,Neutral
deceptive,,,,,,,,,
False,452.546502,3778.415965,0.130773,0.120608,0.121079,0.134827,0.148551,0.120610,0.223552
True,459.625536,6431.186514,0.129634,0.119907,0.120530,0.132197,0.152211,0.119913,0.225608


In [ ]:
df.groupby("deceptive").std()

,id,frame,Angry,Disgust,Fear,Happy,Sad,Surprise,Neutral
deceptive,,,,,,,,,
False,273.434383,3758.070614,0.040251,0.010359,0.012758,0.046865,0.060667,0.010363,0.085442
True,270.189444,2612.202414,0.039913,0.009980,0.013235,0.043731,0.064583,0.009990,0.085189


Minimal difference between truth/lie sequences. The model cannot learn anything, which is shown by the stable loss.